This notebook compares the reflections computed using the zipin model to the reflection computed using ray tracing.

In [1]:
from math import cos, sin, asin, acos, atan2, radians, degrees, sqrt
from vec3 import Vec3
from ray import Ray, reflect
from vgroove import VGroove
from mc import uniform
from stats import histogram
from zipin import zipin
import numpy as np

np.set_printoptions(formatter={'float': '{: 0.3f}'.format})

N = 10000

def vtrace(v,x,i):
    cosi = cos(radians(i))
    sini = sin(radians(i))
    r = Ray( Vec3(x, 0.0, 0.0), Vec3(-sini, 0.0, -cosi) )
    return v.trace(r)

def trace(m,x,i):
    return vtrace(VGroove(m), x, i)

def brdf(m, i):
    vgroove = VGroove(m)
    return [vtrace(vgroove, uniform(-1.0, 1.0), i) for _ in range(N)]

def printbrdf(m,i):
    hits = brdf(m, i)
    hist = histogram(hits)
    print('m={}, i={}'.format(m,i) )
    for r in range(len(hist)):
        h = hist[r,:]
        if sum(h) != 0.:
            print('r={}'.format(r-90), sum(h), h)

def printtrace(m, x, i):
    hits = trace(m, x, i)
    print(len(hits), '{:0.3f}'.format(x), end=' ')
    for hit in hits:
        deg = int(degrees(atan2(hit.r.x,hit.r.z))+0.5)
        print(hit.side, deg, end=' ')
    print()



Let's run a test with a v-groove angle of 45 and ray incident at 30. The correct result is that we have two reflected directions, 30 and 60. The left side generates a 1 bounce reflection at 60, and a 2 bounce reflection at 30. The right side only generates a 2 bounce reflection at 30. The calculated angles of reflection and percentages of each path are calculated using MC ray tracing and using the Zipin model. They match.

In [2]:
phi = 45
theta = 30
#for i in range(20):
#    printtrace(phi, i/10-1, theta)
printbrdf(phi, theta)
print(zipin(radians(phi),radians(theta)))

m=45, i=30
r=30 0.4259 [ 0.000  0.000  0.426  0.000  0.000  0.000  0.000  0.000  0.000  0.000]
r=60 0.5741 [ 0.000  0.574  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000]
[(60.00000000000001, 0.5773502691896256, 1, 'left'), (29.999999999999993, 0.21132486540518736, 2, 'left'), (29.999999999999993, 0.21132486540518716, 2, 'right')]
